# Notebook : Optimisation Avancée des Hyperparamètres pour Random Forest

**Objectifs :**
1. Optimiser les hyperparamètres du modèle **Random Forest** en utilisant **RandomizedSearchCV** pour explorer une plus large gamme de valeurs sans augmenter la taille de l'échantillon (20% des données).
2. Utiliser une **validation croisée** pour évaluer les performances du modèle et vérifier sa stabilité.
3. Vérifier si le modèle atteint le seuil de performance souhaité (R² ≥ 0.95).

---

## Étape 1 : Importer les bibliothèques nécessaires et charger les données

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import os
import json
import numpy as np


---

## Étape 2 : Charger les données et préparer un échantillon de 20%

In [2]:
# Charger les données complètes
file_path = 'note-books/phase_2_training/04_correlation_analysis/data_final_features.csv'
df = pd.read_csv(file_path)

# Encoder `nutriscore_grade` en valeurs numériques
df['nutriscore_grade_encoded'] = df['nutriscore_grade'].map({'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4})

# Préparer les variables explicatives et la cible
X = df[['fat_100g', 'saturated-fat_100g', 'energy_100g', 'energy-kcal_100g', 'sugars_100g']]
y = df['nutriscore_grade_encoded']

# Prendre un échantillon de 20% des données pour l’entraînement
X_sample, _, y_sample, _ = train_test_split(X, y, test_size=0.8, random_state=42)

---

## Étape 3 : Définir la pipeline et les hyperparamètres pour RandomizedSearchCV

Nous définissons une pipeline avec une étape de standardisation et le modèle **Random Forest**, puis nous créons une grille de recherche plus large pour les hyperparamètres

In [3]:
# Créer une pipeline avec une étape de standardisation et le modèle Random Forest
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(random_state=42))
])

# Définir une grille de recherche étendue pour les hyperparamètres
param_dist = {
    'model__n_estimators': [100, 200, 300, 500],
    'model__max_depth': [10, 20, 30, None],
    'model__min_samples_split': [2, 5, 10, 15],
    'model__min_samples_leaf': [1, 2, 4, 6]
}

# Configurer RandomizedSearchCV pour explorer les hyperparamètres
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=20,  # Nombre d'itérations pour limiter le temps d'exécution
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    random_state=42,
    verbose=2
)

# Entraîner le modèle avec Random Search sur l'échantillon de 20% des données
random_search.fit(X_sample, y_sample)

# Extraire les meilleurs paramètres
best_params = random_search.best_params_
print("Meilleurs paramètres trouvés avec Random Search :", best_params)

# Sauvegarder les meilleurs paramètres pour un suivi ultérieur
output_folder = 'note-books/phase_2_training/09_random_forest_advanced_optimization'
os.makedirs(output_folder, exist_ok=True)
best_params_path = f'{output_folder}/best_hyperparameters.json'
with open(best_params_path, 'w') as f:
    json.dump(best_params, f)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END model__max_depth=None, model__min_samples_leaf=4, model__min_samples_split=5, model__n_estimators=100; total time=   5.5s
[CV] END model__max_depth=None, model__min_samples_leaf=4, model__min_samples_split=5, model__n_estimators=100; total time=   5.6s
[CV] END model__max_depth=None, model__min_samples_leaf=4, model__min_samples_split=5, model__n_estimators=100; total time=   5.6s
[CV] END model__max_depth=None, model__min_samples_leaf=4, model__min_samples_split=5, model__n_estimators=100; total time=   5.6s
[CV] END model__max_depth=None, model__min_samples_leaf=4, model__min_samples_split=5, model__n_estimators=100; total time=   5.7s
[CV] END model__max_depth=10, model__min_samples_leaf=1, model__min_samples_split=5, model__n_estimators=300; total time=  11.5s
[CV] END model__max_depth=10, model__min_samples_leaf=1, model__min_samples_split=5, model__n_estimators=300; total time=  11.5s
[CV] END model__max_depth

---

## Étape 4 : Évaluer les performances avec validation croisée

Nous évaluons les performances du modèle optimisé en utilisant une **validation croisée** pour obtenir une estimation robuste de ses performances.

In [4]:
# Utiliser le meilleur modèle trouvé par Random Search
best_model = random_search.best_estimator_

# Évaluer le modèle optimisé avec validation croisée
kfold = 5  # Utiliser 5 plis pour la validation croisée
scores = cross_val_score(best_model, X_sample, y_sample, cv=kfold, scoring='r2')
mean_score = np.mean(scores)
std_dev = np.std(scores)

print(f"Validation croisée sur {kfold} plis - Score R² moyen : {mean_score:.4f}, Écart-type : {std_dev:.4f}")

# Vérification du seuil de performance
if mean_score >= 0.95:
    print("Le modèle atteint le seuil de performance souhaité avec la validation croisée (R² moyen ≥ 0.95).")
else:
    print("Le modèle n'atteint toujours pas le seuil de performance souhaité. Envisagez des options supplémentaires.")


Validation croisée sur 5 plis - Score R² moyen : 0.6593, Écart-type : 0.0063
Le modèle n'atteint toujours pas le seuil de performance souhaité. Envisagez des options supplémentaires.


---

## Étape 5 : Sauvegarder le modèle optimisé et ses paramètres

Si le modèle atteint le seuil de performance, nous sauvegardons cette version pour des étapes ultérieures.

In [5]:
# Chemin de sauvegarde pour le modèle optimisé
model_path_optimized = f'{output_folder}/Random_Forest_20_percent_optimized.joblib'

# Sauvegarder le modèle optimisé
joblib.dump(best_model, model_path_optimized)
print(f"Modèle Random Forest optimisé sauvegardé sous : {model_path_optimized}")

Modèle Random Forest optimisé sauvegardé sous : note-books/phase_2_training/09_random_forest_advanced_optimization/Random_Forest_20_percent_optimized.joblib


---

### Explication des étapes

1. **Maintien de l'échantillon à 20%** : Nous conservons 20% des données pour le modèle afin de limiter la charge de calcul.
2. **Optimisation avec RandomizedSearchCV** : Random Search nous permet d'explorer une large gamme d’hyperparamètres sans augmenter significativement le temps d’entraînement.
3. **Évaluation avec validation croisée** : La validation croisée aide à obtenir des scores de performance robustes et à mesurer la stabilité des performances du modèle.
4. **Vérification de seuil de performance** : Un test de seuil est effectué pour déterminer si le modèle répond aux exigences de performance.

---

### Résumé

Dans ce notebook, nous avons :
1. Utilisé **RandomizedSearchCV** pour optimiser les hyperparamètres du modèle **Random Forest** avec un échantillon de 20%.
2. Évalué les performances du modèle avec validation croisée pour obtenir des scores robustes.
3. Vérifié si le modèle atteint le seuil de performance souhaité et sauvegardé le modèle si les résultats sont satisfaisants.

Si le modèle atteint le niveau de performance souhaité (R² moyen ≥ 0.95), il est prêt pour être déployé ou pour des étapes supplémentaires de validation.